In [ ]:
### This script will display the network in its entirety. 
### If you want you can enable physics for drawing the network
### and display the physics buttons to find a good visualization.
### If you want to visualize a local region of the network you 
### can use the third script.

from pyvis.network import Network
from pathlib import Path
import networkx as nx

folder = Path()

eu_roads_gml_file = folder / "eu_roads.gml"

### Read the gml file as a graph
eu_roads_graph = nx.read_gml(eu_roads_gml_file)

### Create a pyvis network from the graph
eu_roads_network = Network("800px", width="100%", bgcolor="#222222", font_color="white", \
                           heading="EU Roads Network")
eu_roads_network.from_nx(eu_roads_graph)

### Manipulate note attributes for display purposes
for node in eu_roads_network.nodes:
    ### We first set the size of the node to be equal to its degree
    node["size"] = eu_roads_graph.degree[node["id"]]
    ### We then set the title to be the name of the city in bold
    node["title"] = "<b>" + node["city"] + "</b>"
    
### Uncomment the next line to disable the physics for drawing the graph
### eu_roads_network.toggle_physics(False)

### You can control the physics in the output page by uncommenting the next line
### By switching of the "Physics Enabled" box the nodes will stop moving
### eu_roads_network.show_buttons(filter_=['physics'])

eu_roads_network.show("eu_roads.html")

In [ ]:
import random as rnd

def compute_local_neighborhood(G,node,size):
    if (node not in G.nodes):
        idx = int(rnd.uniform(0, len(G.nodes)))
        node = list(G.nodes)[idx]
    
    if (size <= 0):
        size = 2
        
    local_graph = nx.Graph()
    
    
    current_size = 0
    
    current_neighborhood = set()
    next_neighborhood = set()
    
    current_neighborhood.add(node)
    
    while current_size < size:
        current_size += 1
        for node in current_neighborhood:
            neighbors = G.adj[node]
            city = G.nodes[node]["city"]
            local_graph.add_node(node,city=city)
            for neighbor in neighbors:
                city = G.nodes[neighbor]["city"]
                local_graph.add_node(neighbor,city=city)
                local_graph.add_edge(node,neighbor)
                next_neighborhood.add(neighbor)
        
        current_neighborhood = next_neighborhood.copy()
        next_neighborhood = set()
        
    
    return local_graph

In [ ]:
### This script will draw a subnetwork of the entire network. For this
### it will take a node [root_node] and look at the neighborhood up to
### size [size_of_neighborhood]. If this size is 1, it will only show
### the neighbors. For size 2 it will also show all the neighbors of the
### neighbors. And so on. If no root_node is specified a random node will
### be selected that has degree > 0.
### NOTE: do not forget to run the script above so that the function
### compute_local_neighborhood is defined.

from pyvis.network import Network
from pathlib import Path
import networkx as nx
import random as rnd

root_node = ""
size_of_neighborhood = 5

folder = Path()

eu_roads_gml_file = folder / "eu_roads.gml"

### Read the gml file as a graph
eu_roads_graph = nx.read_gml(eu_roads_gml_file)

### If the specified root is not in the graph we select a node with
### degree > 0 at random
if (root_node not in eu_roads_graph.nodes):
    idx = int(rnd.uniform(0, len(eu_roads_graph.nodes)))
    root_node = list(eu_roads_graph.nodes)[idx]
    while (eu_roads_graph.degree[root_node] <= 0):
        idx = int(rnd.uniform(0, len(eu_roads_graph.nodes)))
        root_node = list(eu_roads_graph.nodes)[idx]
        
heading = f"EU Roads Network, Local view from: {root_node} of size {size_of_neighborhood}"

small_eu_roads_graph = compute_local_neighborhood(eu_roads_graph,root_node,size_of_neighborhood)

### Create a pyvis network from the graph
small_eu_roads_network = Network("800px", width="100%", bgcolor="#222222", font_color="white", \
                           heading=heading)
small_eu_roads_network.from_nx(small_eu_roads_graph)

### Manipulate note attributes for display purposes
for node in small_eu_roads_network.nodes:
    if (node["id"] == root_node):
        node.update({'x':0, 'y':0, 'color':'red'})

    ### We set the size of the node to be equal to its degree
    node["size"] = small_eu_roads_graph.degree[node["id"]]
    ### We then set the title to be the name of the city in bold
    node["title"] = "<b>" + node["city"] + "</b>"
    
### Uncomment the next line to disable the physics for drawing the graph
#small_eu_roads_network.toggle_physics(False)

### You can control the physics in the output page by uncommenting the next line
### By switching of the "Physics Enabled" box the nodes will stop moving
#small_eu_roads_network.show_buttons(filter_=['physics'])

small_eu_roads_network.show("eu_roads.html")